In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore') 
import numpy as np

In [2]:
import fbprophet

In [3]:
train = pd.read_csv('02-1_내국인유동인구_KOREAN.CSV', encoding = 'cp949')

In [4]:
train

,base_date,time,city,emd_cd,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,제주시,50110250,한림읍,여성,0,638.5509,15.5400,151.5149,제주
1,2018-01-01,1,제주시,50110250,한림읍,여성,0,0.0000,3.7249,171.3458,그외
2,2018-01-01,1,제주시,50110250,한림읍,여성,10,668.4577,17.0308,166.0500,제주
3,2018-01-01,1,제주시,50110250,한림읍,여성,10,0.0000,4.2577,195.8539,그외
4,2018-01-01,1,제주시,50110250,한림읍,여성,20,609.7710,5.9490,148.7246,제주
...,...,...,...,...,...,...,...,...,...,...,...
47250191,2021-06-30,24,서귀포시,50130620,예래동,남성,20,0.0000,0.0000,599.1969,그외
47250192,2021-06-30,24,서귀포시,50130620,예래동,남성,20,187.5990,41.9998,75.5996,제주
47250193,2021-06-30,24,서귀포시,50130620,예래동,남성,10,121.8467,0.0000,4.5128,제주
47250194,2021-06-30,24,서귀포시,50130620,예래동,남성,80,122.2190,0.0000,4.5032,제주


In [5]:
train1 = train.drop(columns=['time','city','emd_cd','age','resd'])

In [6]:
train1

,base_date,emd_nm,sex,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
0,2018-01-01,한림읍,여성,638.5509,15.5400,151.5149
1,2018-01-01,한림읍,여성,0.0000,3.7249,171.3458
2,2018-01-01,한림읍,여성,668.4577,17.0308,166.0500
3,2018-01-01,한림읍,여성,0.0000,4.2577,195.8539
4,2018-01-01,한림읍,여성,609.7710,5.9490,148.7246
...,...,...,...,...,...,...
47250191,2021-06-30,예래동,남성,0.0000,0.0000,599.1969
47250192,2021-06-30,예래동,남성,187.5990,41.9998,75.5996
47250193,2021-06-30,예래동,남성,121.8467,0.0000,4.5128
47250194,2021-06-30,예래동,남성,122.2190,0.0000,4.5032


In [7]:
train2 = train1.groupby(['base_date','emd_nm','sex']).agg(sum).reset_index()

In [8]:
visitor = train1[train['resd']=='그외']

In [9]:
visitor1 = visitor.groupby(['base_date','emd_nm','sex']).agg(sum).reset_index()

In [10]:
visitor1

,base_date,emd_nm,sex,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
0,2018-01-01,건입동,남성,0.0,1625.4836,32026.2855
1,2018-01-01,건입동,여성,0.0,749.7915,28371.7287
2,2018-01-01,구좌읍,남성,0.0,2687.7942,56984.3962
3,2018-01-01,구좌읍,여성,0.0,1767.4462,69723.5401
4,2018-01-01,남원읍,남성,0.0,2821.0299,51634.7730
...,...,...,...,...,...,...
109817,2021-06-30,한림읍,여성,0.0,2886.5435,103271.2751
109818,2021-06-30,화북동,남성,0.0,3842.1309,25778.2446
109819,2021-06-30,화북동,여성,0.0,1922.6874,21208.4264
109820,2021-06-30,효돈동,남성,0.0,545.6342,8326.4054


In [11]:
outsider_man = visitor1[visitor1['sex']=='남성']
outsider_woman = visitor1[visitor1['sex']=='여성']

In [12]:
outsider_man1 = outsider_man.drop(columns =['sex'])
outsider_woman1 = outsider_woman.drop(columns =['sex'])

In [13]:
outsider_woman2 = outsider_woman1.rename(columns = {'resd_pop_cnt':'outwoman_resd_pop_cnt','work_pop_cnt':'outwoman_work_pop_cnt','visit_pop_cnt':'outwoman_visit_pop_cnt'})

In [14]:
outsider_man2 = outsider_man1.rename(columns = {'resd_pop_cnt':'outman_resd_pop_cnt','work_pop_cnt':'outman_work_pop_cnt','visit_pop_cnt':'outman_visit_pop_cnt'})

In [15]:
man = train2[train2['sex'] == '남성'].reset_index()

In [16]:
girl = train2[train2['sex'] == '여성'].reset_index()

In [17]:
girl

,index,base_date,emd_nm,sex,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
0,1,2018-01-01,건입동,여성,85324.4591,7335.6288,86268.1594
1,3,2018-01-01,구좌읍,여성,126427.3259,7106.2017,126403.5696
2,5,2018-01-01,남원읍,여성,132073.3097,6727.0772,105680.3226
3,7,2018-01-01,노형동,여성,604264.9052,36706.3146,203360.6607
4,9,2018-01-01,대륜동,여성,139747.2482,8660.3649,107820.8461
...,...,...,...,...,...,...,...
54906,109813,2021-06-30,표선면,여성,115466.4421,9502.4429,75581.9336
54907,109815,2021-06-30,한경면,여성,78815.2963,7237.9656,45812.9111
54908,109817,2021-06-30,한림읍,여성,222983.3985,17722.0108,138819.5661
54909,109819,2021-06-30,화북동,여성,295212.0508,19883.9228,102795.4549


In [18]:
man1 = man.rename(columns = {'resd_pop_cnt':'man_resd_pop_cnt','work_pop_cnt':'man_work_pop_cnt','visit_pop_cnt':'man_visit_pop_cnt'})

In [19]:
girl1 = girl.rename(columns = {'resd_pop_cnt':'woman_resd_pop_cnt','work_pop_cnt':'woman_work_pop_cnt','visit_pop_cnt':'woman_visit_pop_cnt'})

In [20]:
alldata= pd.merge(man1,girl1, on=['base_date','emd_nm'],how='left')

In [21]:
alldata

,index_x,base_date,emd_nm,sex_x,man_resd_pop_cnt,man_work_pop_cnt,man_visit_pop_cnt,index_y,sex_y,woman_resd_pop_cnt,woman_work_pop_cnt,woman_visit_pop_cnt
0,0,2018-01-01,건입동,남성,84744.8701,11638.6479,95737.5602,1,여성,85324.4591,7335.6288,86268.1594
1,2,2018-01-01,구좌읍,남성,142664.8188,9721.0341,116039.1218,3,여성,126427.3259,7106.2017,126403.5696
2,4,2018-01-01,남원읍,남성,152384.8823,10995.4261,108160.7162,5,여성,132073.3097,6727.0772,105680.3226
3,6,2018-01-01,노형동,남성,567232.7968,35126.5886,221249.7767,7,여성,604264.9052,36706.3146,203360.6607
4,8,2018-01-01,대륜동,남성,124968.7721,11301.7721,100900.0818,9,여성,139747.2482,8660.3649,107820.8461
...,...,...,...,...,...,...,...,...,...,...,...,...
54906,109812,2021-06-30,표선면,남성,123817.9381,14018.5155,82469.6690,109813,여성,115466.4421,9502.4429,75581.9336
54907,109814,2021-06-30,한경면,남성,88602.5648,8791.6569,52753.3357,109815,여성,78815.2963,7237.9656,45812.9111
54908,109816,2021-06-30,한림읍,남성,247501.5803,31471.0957,148131.6335,109817,여성,222983.3985,17722.0108,138819.5661
54909,109818,2021-06-30,화북동,남성,277288.5428,30268.6911,138252.9350,109819,여성,295212.0508,19883.9228,102795.4549


In [22]:
alldata1 = alldata.drop(columns = ['sex_x','index_x','index_y','sex_y'])

In [23]:
outsider = pd.merge(outsider_man2,outsider_woman2, on =['base_date','emd_nm'] , how='left')

In [24]:
outsider

,base_date,emd_nm,outman_resd_pop_cnt,outman_work_pop_cnt,outman_visit_pop_cnt,outwoman_resd_pop_cnt,outwoman_work_pop_cnt,outwoman_visit_pop_cnt
0,2018-01-01,건입동,0.0,1625.4836,32026.2855,0.0,749.7915,28371.7287
1,2018-01-01,구좌읍,0.0,2687.7942,56984.3962,0.0,1767.4462,69723.5401
2,2018-01-01,남원읍,0.0,2821.0299,51634.7730,0.0,1326.8274,55675.7184
3,2018-01-01,노형동,0.0,5242.3771,67799.0845,0.0,3364.4980,66692.4214
4,2018-01-01,대륜동,0.0,2117.9781,42642.1220,0.0,1146.0216,51628.4218
...,...,...,...,...,...,...,...,...
54906,2021-06-30,표선면,0.0,2775.1146,50185.5103,0.0,2016.1363,54881.0293
54907,2021-06-30,한경면,0.0,1637.2200,26608.2573,0.0,1045.9064,30097.5739
54908,2021-06-30,한림읍,0.0,5922.3840,81217.2597,0.0,2886.5435,103271.2751
54909,2021-06-30,화북동,0.0,3842.1309,25778.2446,0.0,1922.6874,21208.4264


In [25]:
alldata2 =  pd.merge(alldata1, outsider, on =['base_date','emd_nm'] , how='left')

In [26]:
alldata3 = alldata2.drop(columns = ['outman_resd_pop_cnt','outwoman_resd_pop_cnt'])

In [29]:
alldata3

,base_date,emd_nm,man_resd_pop_cnt,man_work_pop_cnt,man_visit_pop_cnt,woman_resd_pop_cnt,woman_work_pop_cnt,woman_visit_pop_cnt,outman_work_pop_cnt,outman_visit_pop_cnt,outwoman_work_pop_cnt,outwoman_visit_pop_cnt
0,2018-01-01,건입동,84744.8701,11638.6479,95737.5602,85324.4591,7335.6288,86268.1594,1625.4836,32026.2855,749.7915,28371.7287
1,2018-01-01,구좌읍,142664.8188,9721.0341,116039.1218,126427.3259,7106.2017,126403.5696,2687.7942,56984.3962,1767.4462,69723.5401
2,2018-01-01,남원읍,152384.8823,10995.4261,108160.7162,132073.3097,6727.0772,105680.3226,2821.0299,51634.7730,1326.8274,55675.7184
3,2018-01-01,노형동,567232.7968,35126.5886,221249.7767,604264.9052,36706.3146,203360.6607,5242.3771,67799.0845,3364.4980,66692.4214
4,2018-01-01,대륜동,124968.7721,11301.7721,100900.0818,139747.2482,8660.3649,107820.8461,2117.9781,42642.1220,1146.0216,51628.4218
...,...,...,...,...,...,...,...,...,...,...,...,...
54906,2021-06-30,표선면,123817.9381,14018.5155,82469.6690,115466.4421,9502.4429,75581.9336,2775.1146,50185.5103,2016.1363,54881.0293
54907,2021-06-30,한경면,88602.5648,8791.6569,52753.3357,78815.2963,7237.9656,45812.9111,1637.2200,26608.2573,1045.9064,30097.5739
54908,2021-06-30,한림읍,247501.5803,31471.0957,148131.6335,222983.3985,17722.0108,138819.5661,5922.3840,81217.2597,2886.5435,103271.2751
54909,2021-06-30,화북동,277288.5428,30268.6911,138252.9350,295212.0508,19883.9228,102795.4549,3842.1309,25778.2446,1922.6874,21208.4264


In [27]:
alldata3.to_csv('내국인_유동인구.csv', index = False)